In [1]:
import sys
sys.path.append('..')

from datasrcs.spc import load_full_tors
from regions import load_region, CHASE_ALLEY
from reanalysis import ncep_r1
from plotting.mapareas import conus

%env WORKDIR=~/weatherpy-work

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
%matplotlib inline

from datetime import datetime

env: WORKDIR=~/weatherpy-work


In [2]:
tors = load_full_tors(to_tz='CST')

In [3]:
init_year = 1985
fin_year = tors.yr.max()

In [4]:
latemay_tors = tors[(tors.dy > 15) & (tors.mo == 5)]

chasealleydf, chasealley = load_region(CHASE_ALLEY)
chase_tors = latemay_tors.geospatial.filter_region(chasealley, ['slat', 'slon'])
chase_tors = chase_tors[(chase_tors.yr >= 1985) & (chase_tors.date_time.dt.hour > 12)]

In [5]:
chase_tors.tail()

,date_time,om,yr,mo,dy,tz,st,stf,stn,mag,...,len,wid,ns,sn,sg,f1,f2,f3,f4,fc
62921,2018-05-29 18:25:00,616301,2018,5,29,3,OK,40,0,1,...,2.50,20,1,1,1,151,0,0,0,0
62922,2018-05-29 19:12:00,616302,2018,5,29,3,KS,20,0,0,...,0.30,30,1,1,1,157,0,0,0,0
62928,2018-05-31 13:43:00,616308,2018,5,31,3,ND,38,0,0,...,0.32,50,1,1,1,25,0,0,0,0
62930,2018-05-31 16:19:00,616310,2018,5,31,3,ND,38,0,0,...,0.42,50,1,1,1,89,0,0,0,0
62932,2018-05-31 17:15:00,616312,2018,5,31,3,ND,38,0,0,...,0.25,50,1,1,1,59,0,0,0,0


In [6]:
tors_agg = chase_tors.groupby(['yr', 'mo', 'dy']).count()['date_time']
sigtor_agg = chase_tors[chase_tors.mag >= 2].groupby(['yr', 'mo', 'dy']).count()['date_time']

In [7]:
tordays = pd.concat([
    tors_agg[tors_agg >= 3],
    sigtor_agg
], axis=1)

In [8]:
tordays.shape

(199, 2)

In [9]:
tordays.tail()

date_time  date_time
yr   mo dy                      
2018 5  19        3.0        NaN
        25        4.0        NaN
        28       23.0        NaN
        29        6.0        NaN
        31        3.0        NaN

In [10]:
def has_tors(yr, mo, dy):
    q ='yr == @yr & mo == @mo & dy == @dy'
    tordays_queried = tordays.query(q)
    return tordays_queried.shape[0] > 0

In [11]:
datemap = {}
for yr in range(init_year, fin_year + 1):
    month = 5
    for dy in range(16, 32):
        datemap[datetime(yr, month, dy)] = has_tors(yr, month, dy)

In [20]:
tordays = pd.DataFrame([datemap]).T.reset_index()
tordays.columns = ['date', 'tor']

In [21]:
tordays.head()

,date,tor
0,1985-05-16,False
1,1985-05-17,False
2,1985-05-18,False
3,1985-05-19,False
4,1985-05-20,True


In [22]:
# tordays.to_csv('tordays_chasealley.csv', index=False)